****Scraping Tables From Multiple pages****

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://www.scrapethissite.com/pages/forms/?page_num=1&per_page=100"

In [3]:
page = requests.get(url)

In [4]:
page

<Response [200]>

In [5]:
print(page.text)

<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>
    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png" />

    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="description" content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components.">

    <link href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" rel="stylesheet" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" crossorigin="anonymous">
    <link href='https://fonts.googleapis.com/css?family=Lato:400,700' rel='stylesheet' type='text/css'>
    <link rel="stylesheet" type="text/css" href="/static/css/styles.css">

   

In [6]:
soup = BeautifulSoup(page.text,'html')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping
  </title>
  <link href="/static/images/scraper-icon.png" rel="icon" type="image/png"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface components." name="description"/>
  <link crossorigin="anonymous" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.5/css/bootstrap.min.css" integrity="sha256-MfvZlkHCEqatNoGiOXveE8FIwMzZg4W85qfrfIFBfYc= sha512-dTfge/zgoMYpP7QbHy4gWMEGsbsdZeCXz7irItjcC3sPUFtf0kuFbDz/ixG7ArTxmDjLXDmezHubeNikyKGVyQ==" rel="stylesheet"/>
  <link href="https://fonts.googleapis.com/css?family=Lato:400,700" rel="stylesheet" type="text/css"/>
  <link href="/static/css/styles.css" rel="stylesheet" type="text/css"/>
  <meta con

In [7]:
# To store all tables in a list of DataFrames
all_tables = []

# Function to get the total number of pages
def get_total_pages():
    response = requests.get(url.format(1))
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find the pagination section
    pagination = soup.find("ul", class_="pagination") #ul class="pagination"
    
    # Check if pagination exists
    if pagination:
        # Get the total number of pages from the last page in the pagination
        page_numbers = pagination.find_all("a") # a refers to the links of pages one by one
        if page_numbers:
            last_page = page_numbers[-2].text  # The second to last "a" element should be the last page number
            return int(last_page)
    return 1  # Default to 1 if no pagination is found

# Loop through all pages
total_pages = get_total_pages()
print(f"Total pages: {total_pages}")

for page_num in range(1, total_pages + 1):
    # Send request for the current page
    response = requests.get(url.format(page_num))
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    for table in tables:
        # Use pandas to directly read the table into DataFrame
        df = pd.read_html(str(table))[0]  # Convert the HTML table to DataFrame
        all_tables.append(df)  # Append the DataFrame to the list

# Now all_tables contains a list of DataFrames, one for each table
# You can optionally concatenate all tables into one DataFrame
combined_df = pd.concat(all_tables, ignore_index=True)

# Print combined DataFrame or save it to a CSV
print(combined_df)

# Optionally save to CSV
combined_df.to_csv('all_tables_data.csv', index=False)

Total pages: 6


C:\Users\Noreen\AppData\Local\Temp\ipykernel_8344\1017803319.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Convert the HTML table to DataFrame
C:\Users\Noreen\AppData\Local\Temp\ipykernel_8344\1017803319.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Convert the HTML table to DataFrame
C:\Users\Noreen\AppData\Local\Temp\ipykernel_8344\1017803319.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Convert the HTML table to DataFrame
C:\Users\Noreen\AppData\Local\Temp\ipykernel_8344\1017803319.py:35:

              Team Name  Year  Wins  Losses  OT Losses  Win %  Goals For (GF)  \
0         Boston Bruins  1990    44      24        NaN  0.550             299   
1        Buffalo Sabres  1990    31      30        NaN  0.388             292   
2        Calgary Flames  1990    46      26        NaN  0.575             344   
3    Chicago Blackhawks  1990    49      23        NaN  0.613             284   
4     Detroit Red Wings  1990    34      38        NaN  0.425             273   
..                  ...   ...   ...     ...        ...    ...             ...   
595      Buffalo Sabres  1994    22      19        NaN  0.458             130   
596      Calgary Flames  1994    24      17        NaN  0.500             163   
597  Chicago Blackhawks  1994    24      19        NaN  0.500             156   
598        Dallas Stars  1994    17      23        NaN  0.354             136   
599   Detroit Red Wings  1994    33      11        NaN  0.688             180   

     Goals Against (GA)  + 

C:\Users\Noreen\AppData\Local\Temp\ipykernel_8344\1017803319.py:35: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # Convert the HTML table to DataFrame
